## Phase 2: Model Conversion

This notebook converts the custom trained .h5 models into edge-optimized formats.

*   TensorFlow.js format (.json + .bin) for use in a Progressive Web App (PWA)
*   TensorFlow Lite format (.tflite) for use in mobile apps

This ensures models are ready for execution on-device, enabling real-time predictions without server calls.


**1. Setting up the environment**

In [ ]:
!pip install -q tensorflow tensorflowjs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 2025.7.1 requires packaging>=24.1, but you have packaging 23.2 which is incompatible.
google-cloud-bigquery 3.35.1 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.
db-dtypes 1.4.3 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.


In [9]:
import os
import tensorflow as tf
import tensorflowjs as tfjs
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd

In [10]:
# define paths
from google.colab import drive
drive.mount('/content/drive')

# ✅ Correct base path based on your Drive structure
BASE_PATH = "/content/drive/My Drive/Edge_AI_Project/models"
H5_PATH = os.path.join(BASE_PATH, "saved_models")
TFJS_PATH = os.path.join(BASE_PATH, "tfjs_models")
TFLITE_PATH = os.path.join(BASE_PATH, "tflite_models")
LOG_PATH = os.path.join(BASE_PATH, "conversion_logs")

os.makedirs(TFJS_PATH, exist_ok=True)
os.makedirs(TFLITE_PATH, exist_ok=True)
os.makedirs(LOG_PATH, exist_ok=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**2. Model Conversion Functions**

In [11]:
# TF.js Conversion
def convert_to_tfjs(h5_model_path, output_dir):
    tfjs.converters.save_keras_model(load_model(h5_model_path), output_dir)


In [12]:
# TFLite Conversion with Quantization
def convert_to_tflite(h5_model_path, output_path):
    model = load_model(h5_model_path)
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Post-training quantization
    tflite_model = converter.convert()

    with open(output_path, 'wb') as f:
        f.write(tflite_model)


**3. Conversion and Logging Loop**

In [13]:
# Model Metadata
models = {
    "age":    {"filename": "age_model.h5",    "input_shape": (224, 224, 3)},
    "gender": {"filename": "gender_model.h5", "input_shape": (224, 224, 3)},
    "emotion": {"filename": "emotion_model.h5", "input_shape": (48, 48, 1)},
}

log_data = []

for name, config in models.items():
    print(f"🔄 Converting {name} model...")

    h5_path = os.path.join(H5_PATH, config["filename"])
    tfjs_dir = os.path.join(TFJS_PATH, name)
    tflite_path = os.path.join(TFLITE_PATH, f"{name}.tflite")

    # Convert to TensorFlow.js
    convert_to_tfjs(h5_path, tfjs_dir)

    # Convert to TFLite
    convert_to_tflite(h5_path, tflite_path)

    # Logging
    tfjs_size = sum(os.path.getsize(os.path.join(tfjs_dir, f)) for f in os.listdir(tfjs_dir))
    tflite_size = os.path.getsize(tflite_path)
    h5_size = os.path.getsize(h5_path)

    model = load_model(h5_path)

    log_data.append({
        "Model": name.capitalize(),
        "Input Shape": config["input_shape"],
        ".h5 Size (KB)": round(h5_size / 1024, 2),
        "TF.js Size (KB)": round(tfjs_size / 1024, 2),
        ".tflite Size (KB)": round(tflite_size / 1024, 2),
        "Output .tflite": tflite_path,
        "Output TF.js Dir": tfjs_dir
    })


🔄 Converting age model...


failed to lookup keras version from the file,
    this is likely a weight only file


Saved artifact at '/tmp/tmpd58qjwab'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  131998255499792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998083861584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998083858704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998194170000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998083862352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998083863888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998083864272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998083864656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998083864464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998083861200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998083865808

🔄 Converting gender model...


failed to lookup keras version from the file,
    this is likely a weight only file


Saved artifact at '/tmp/tmpypkxz53e'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  131998077363280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998077368080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998077368464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998077368272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998077367696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998077369616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998077370192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998077368848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998077370000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998077367120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998075371792

🔄 Converting emotion model...


failed to lookup keras version from the file,
    this is likely a weight only file
Saved artifact at '/tmp/tmpkwgri99n'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 48, 48, 1), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  131998070747792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998070746256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998070742416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998070748944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998070743952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998070748176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998070750672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998070749904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131998243013264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1

**4. Save logs to CSV**

In [14]:
# Save Conversion Logs
df = pd.DataFrame(log_data)
log_file = os.path.join(LOG_PATH, "conversion_summary.csv")
df.to_csv(log_file, index=False)
df


,Model,Input Shape,.h5 Size (KB),TF.js Size (KB),.tflite Size (KB),Output .tflite,Output TF.js Dir
0,Age,"(224, 224, 3)",26885.20,8979.97,2443.16,/content/drive/My Drive/Edge_AI_Project/models...,/content/drive/My Drive/Edge_AI_Project/models...
1,Gender,"(224, 224, 3)",26885.20,8979.97,2443.16,/content/drive/My Drive/Edge_AI_Project/models...,/content/drive/My Drive/Edge_AI_Project/models...
2,Emotion,"(48, 48, 1)",25666.21,8548.44,2142.49,/content/drive/My Drive/Edge_AI_Project/models...,/content/drive/My Drive/Edge_AI_Project/models...


In [ ]:
import IPython.display as display
display.display(df)